In [3]:
# 该模块封装 函数
import pymysql
import re
from sqlalchemy import create_engine
import pandas as pd
import time
import numpy as np
import threading
from queue import Queue
import ipynb_importer
from config import get_mysql_cfg, get_table_cfg, cfg
from typing import List


createVar, timeday, sql_queue = locals(), time.localtime(), Queue()


def main(mysql_cfg: str, table_source: str):
    # 全局变量名,时间元组
    engine = create_engine(get_mysql_cfg(mysql_cfg))  # 链接数据库
    table_cfg_dict = get_table_cfg(table_source)
    for table_name, field_list in table_cfg_dict.items():
        if len(field_list):
            fields = ",".join(field_list)
        else:
            fields = "*"
        sql = "select {} from {}".format(fields, table_name)
        sql_queue.put((sql, table_name))
    thread_list = []
    for i in range(4):
        thread = threading.Thread(target=read_mysql_table, args=(engine,))
        thread_list.append(thread)
    for thread in thread_list:
        thread.start()
    for thread in thread_list:
        thread.join()


# 读取mysql数据
def read_mysql_table(engine):
    while True:
        if sql_queue.empty():
            return
        try:
            sql, table_name = sql_queue.get()
            df = pd.read_sql_query(sql, engine)
            df = need_dtype(df)
            createVar["df_"+table_name] = df
        except Exception as e:
            print("%s 查询错误: %s" % (table_name, str(e)))
            continue

# 转换dataframe数据类型,可以明确指定那些列不需要转换为指定的类型,首先将dataframe中所有可以转为数值数据的转为数字类型,其他转为时间类型


def need_dtype(df: pd.core.frame.DataFrame):
    # 此处应该加逻辑 1.指定那些列需要不能转为数字类型而应该转为 时间类型
    for col_name in df.columns:
        df[col_name] = df[col_name].where(df[col_name].apply(lambda order_name: False if re.match(
            r'(\s+$)|(nan$)', str(order_name), re.I) else True), np.nan)
        try:
            count_number = 0
            # 匹配数字 并且这一列转为数值类型  不能匹配六位数字(六位数字可能为时间类型,
            # 此处排除时间戳形式 1-5位整数必须全为数字加$ 7到无穷位整数必须全为数字加$ 存在小数结尾,增加一列标记位)
            flag = 'flag'
            df[flag] = df[col_name].apply(lambda order_name: True if re.match(
                r'(-?\d{1,5}$)|(-?\d{7,}$)|(-?\d+\.\d+$)', str(order_name), re.I) else False)
            # #获取形状 如果形状大于元数据的形状的1/4 我认为该列可以转为数值类型
            if df[df[flag]].shape[0] > df.shape[0]/4:
                df[col_name] = pd.to_numeric(
                    df[col_name], errors='ignore')  # ignore 无效的解析将返回输入
                df.drop(columns=[flag], inplace=True)
                continue
            df[col_name] = pd.to_datetime(
                df[col_name], errors='ignore')
            df.drop(columns=[flag], inplace=True)
        except:
            continue

    return df


# 获取星级计算
def get_star(x):
    if x['月下单用户数'] >= 50 and x['nmv'] >= 10000 and x['生鲜件数占比'] >= 0.5 and x['月营业天数'] >= (timeday[2]-2):
        return 5
    elif x['月下单用户数'] >= 25 and x['nmv'] >= 3750 and x['月营业天数'] >= (timeday[2]-5):
        return 4
    elif x['月下单用户数'] >= 10 and x['nmv'] >= 1000:
        return 3
    elif x['月下单用户数'] >= 3 and x['nmv'] >= 200:
        return 2
    else:
        return 1

# 转置df对象


def transpose_df(df: pd.core.frame.DataFrame, by_name: str):
    transpose_df_list = []
    for group_name, df_group in df.groupby(by=by_name):
        df_group = df_group.drop(columns=by_name).T
        by_tc = "TC仓"
        for col in df_group.columns:  # 遍历df_group 中每一列,获取列的名称
            df_col = df_group.loc[:, [col]]
            tc_name, df_tc = df_col.loc[by_tc, col], df_col.drop(
                index=by_tc)  # 将当前TC仓的值取出,减少操作
            df_tc.columns = [1]
            df_tc[by_name], df_tc[by_tc], df_tc['指标'] = group_name, tc_name, df_tc.index
            # 设置 蜂窝,TC仓为索引
            df_tc.set_index([by_name, by_tc], inplace=True,
                            drop=False)  # 不保留蜂窝以及TC仓索引请将drop置为True
            df_tc = df_tc[[by_name, by_tc, '指标', 1]]
            transpose_df_list.append(df_tc)
    df = pd.concat(transpose_df_list)  # 数据拼接
    return df


if __name__ == "__main__":
    #     df = transpose_df(df, by_name='蜂窝')
    main("test", "table_pass")
#     df = pd.read_excel(r'E:\钉钉excel文件\cms262.xlsx')